In [1]:
import datetime
import pandas as pd
import hopsworks
import os
from dotenv import load_dotenv
from functions import utils
import json

import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
from functions import utils
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
load_dotenv()
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")

os.environ["HOPSWORKS_API_KEY"] = HOPSWORKS_API_KEY
print(HOPSWORKS_API_KEY)

AhdA5UndBaVILBvJ.cvcBxXmN8mo8HPjR1Ryb7XncvNQ6enGsnTHL3LZIUJJ75mpRD0tTptadPTgx4zRu


In [3]:
CITY = "dublin"
STATION = "HEUSTON BRIDGE (NORTH)"

## Read the bike data from csv-files

In [4]:
# get directory path and all files in it
df_bike = pd.DataFrame()
for file in os.listdir(CITY):
    if file.endswith(".csv"):
        df_tmp = pd.read_csv(f"{CITY}/" + file,  parse_dates=['datetime'], skipinitialspace=True)
        df_tmp['station'] = file.split(".")[0]
        df_bike = pd.concat([df_bike, df_tmp])

df_bike['datetime'] = pd.to_datetime(df_bike['datetime'], utc=True).dt.floor('H') + datetime.timedelta(hours=1)
df_bike = df_bike.set_index(['datetime', 'station']).reset_index(drop=False)

df_bike

,datetime,station,num_bikes_available
0,2025-01-08 15:00:00+00:00,AVONDALE_ROAD,15
1,2025-01-08 14:00:00+00:00,AVONDALE_ROAD,18
2,2025-01-08 13:00:00+00:00,AVONDALE_ROAD,16
3,2025-01-08 12:00:00+00:00,AVONDALE_ROAD,16
4,2025-01-08 11:00:00+00:00,AVONDALE_ROAD,18
...,...,...,...
1057801,2023-08-07 06:00:00+00:00,YORK_STREET_WEST,20
1057802,2023-08-07 05:00:00+00:00,YORK_STREET_WEST,20
1057803,2023-08-07 04:00:00+00:00,YORK_STREET_WEST,21
1057804,2023-08-07 03:00:00+00:00,YORK_STREET_WEST,21


In [5]:
df_bike.dropna(inplace=True)
df_bike["num_bikes_available"] = df_bike["num_bikes_available"].astype("float32")

# Remove all rows that are not the given streets
df_bike = df_bike[df_bike['station'].isin([STATION.replace(" ", "_")])]
df_bike

,datetime,station,num_bikes_available
491787,2025-01-08 15:00:00+00:00,HEUSTON_BRIDGE_(NORTH),39.0
491788,2025-01-08 14:00:00+00:00,HEUSTON_BRIDGE_(NORTH),39.0
491789,2025-01-08 13:00:00+00:00,HEUSTON_BRIDGE_(NORTH),39.0
491790,2025-01-08 12:00:00+00:00,HEUSTON_BRIDGE_(NORTH),38.0
491791,2025-01-08 11:00:00+00:00,HEUSTON_BRIDGE_(NORTH),39.0
...,...,...,...
501061,2023-08-07 06:00:00+00:00,HEUSTON_BRIDGE_(NORTH),10.0
501062,2023-08-07 05:00:00+00:00,HEUSTON_BRIDGE_(NORTH),10.0
501063,2023-08-07 04:00:00+00:00,HEUSTON_BRIDGE_(NORTH),10.0
501064,2023-08-07 03:00:00+00:00,HEUSTON_BRIDGE_(NORTH),10.0


## Get the weather data from Open Mateo

First, we need to get the earliest date of the bike data so that we can get the weater data from Open Mateo for the same period.

In [6]:
earliest_bike_date_raw = pd.Series.min(df_bike['datetime'])
earliest_bike_date = earliest_bike_date_raw.strftime('%Y-%m-%d')
earliest_bike_datetime = earliest_bike_date_raw.strftime('%Y-%m-%d %H:%M:%S %Z')
last_bike_date_raw = pd.Series.max(df_bike['datetime'])
last_bike_date = last_bike_date_raw.strftime('%Y-%m-%d')
last_bike_datetime = last_bike_date_raw.strftime('%Y-%m-%d %H:%M:%S %Z')

earliest_bike_datetime, last_bike_datetime

('2023-08-07 02:00:00 UTC', '2025-01-08 15:00:00 UTC')

Then, we get the weather data from Open Mateo

In [7]:
weather_df = utils.get_historical_weather(CITY, earliest_bike_date, last_bike_date)
weather_df = weather_df.rename(columns={'date_x': 'datetime'})
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'], utc=True).dt.floor('H')
weather_df.dropna(inplace=True)

# remove col date_y, date_only
weather_df = weather_df.drop(columns=['date_y', 'date_only'])

weather_df

Coordinates 53.32161331176758°N -6.501922607421875°E
Elevation 11.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,datetime,temperature_2m,apparent_temperature,rain,snowfall,wind_speed_10m,daylight_duration,rain_sum,city
0,2023-08-07 00:00:00+00:00,11.539500,10.177939,0.0,0.0,8.654986,55149.910156,0.1,dublin
1,2023-08-07 01:00:00+00:00,11.039500,9.729592,0.0,0.0,8.396570,55149.910156,0.1,dublin
2,2023-08-07 02:00:00+00:00,10.439500,8.994471,0.0,0.0,8.942214,55149.910156,0.1,dublin
3,2023-08-07 03:00:00+00:00,10.139501,8.546630,0.0,0.0,9.673221,55149.910156,0.1,dublin
4,2023-08-07 04:00:00+00:00,9.939500,8.157264,0.0,0.0,10.691453,55149.910156,0.1,dublin
...,...,...,...,...,...,...,...,...,...
12451,2025-01-06 19:00:00+00:00,0.439500,-5.516075,0.0,0.0,24.412407,27908.451172,0.5,dublin
12452,2025-01-06 20:00:00+00:00,-0.010500,-6.014175,0.0,0.0,24.056108,27908.451172,0.5,dublin
12453,2025-01-06 21:00:00+00:00,-0.260500,-6.202991,0.0,0.0,23.197662,27908.451172,0.5,dublin
12454,2025-01-06 22:00:00+00:00,-0.260500,-6.153846,0.0,0.0,22.689283,27908.451172,0.5,dublin


## Data validation rules

First, we validate that the number of bikes avaliable is always greater than or equal to zero.

In [8]:
if 'num_bikes_available' not in df_bike.columns:
    raise ValueError("The column 'num_bikes_available' does not exist in the dataframe.")

invalid_rows = df_bike[(df_bike['num_bikes_available'] < 0) | (df_bike['num_bikes_available'] > 100)]

if not invalid_rows.empty:
    print("Validation failed: Some rows have invalid values in 'num_bikes_available'.")
    print(invalid_rows)
    assert False
else:
    print("Validation passed: All values in 'num_bikes_available' are within the range 0 to 100.")
    assert True

Validation passed: All values in 'num_bikes_available' are within the range 0 to 100.


Then, we validate that the rainfall, snowfall, and wind speed are always greater than or equal to zero, and below 1000.

In [9]:
if 'rain' not in weather_df.columns:
    raise ValueError("The column 'rain' does not exist in the dataframe.")
if 'snowfall' not in weather_df.columns:
    raise ValueError("The column 'snowfall' does not exist in the dataframe.")
if 'wind_speed_10m' not in weather_df.columns:
    raise ValueError("The column 'wind_speed_10m' does not exist in the dataframe.")

invalid_rows = weather_df[(weather_df['rain'] < 0) | (weather_df['rain'] > 1000) |
                          (weather_df['snowfall'] < 0) | (weather_df['snowfall'] > 1000) |
                          (weather_df['wind_speed_10m'] < 0) | (weather_df['wind_speed_10m'] > 1000)]

if not invalid_rows.empty:
    print("Validation failed: Some rows have invalid values in the specified columns.")
    print(invalid_rows)
    assert False
else:
    print("Validation passed: All values in the specified columns are within the range 0 to 1000.")
    assert True


Validation passed: All values in the specified columns are within the range 0 to 1000.


## Connect to Hopsworks and create two feature groups (+more)

In [10]:
project = hopsworks.login()
fs = project.get_feature_store()

2025-01-21 10:48:34,861 INFO: Initializing external client
2025-01-21 10:48:34,863 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-21 10:48:36,976 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207494


Save the city location and times for the data to Hopsworks for easy access later.

In [11]:
"""
latitude, longitude = utils.get_city_coordinates(CITY)

city_secrets_dict = {
    "city": CITY,
    "latitude": latitude,
    "longitude": longitude,
}

time_secrets_dict = {
    "earliest_bike_datetime": earliest_bike_datetime,
    "last_bike_datetime": last_bike_datetime
}

city_secrets_json = json.dumps(city_secrets_dict)
time_secrets_json = json.dumps(time_secrets_dict)

try:
    secrets.create_secret("city_secrets", city_secrets_json)
except hopsworks.RestAPIError:
    print("city_secrets already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")
    existing_key = secrets.get_secret("city_secrets").value
    print(f"{existing_key}")
    print()

try:
    secrets.create_secret("time_secrets", time_secrets_json)
except hopsworks.RestAPIError:
    print("time_secrets already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")
    existing_key = secrets.get_secret("time_secrets").value
    print(f"{existing_key}")
"""


'\nlatitude, longitude = utils.get_city_coordinates(CITY)\n\ncity_secrets_dict = {\n    "city": CITY,\n    "latitude": latitude,\n    "longitude": longitude,\n}\n\ntime_secrets_dict = {\n    "earliest_bike_datetime": earliest_bike_datetime,\n    "last_bike_datetime": last_bike_datetime\n}\n\ncity_secrets_json = json.dumps(city_secrets_dict)\ntime_secrets_json = json.dumps(time_secrets_dict)\n\ntry:\n    secrets.create_secret("city_secrets", city_secrets_json)\nexcept hopsworks.RestAPIError:\n    print("city_secrets already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")\n    existing_key = secrets.get_secret("city_secrets").value\n    print(f"{existing_key}")\n    print()\n\ntry:\n    secrets.create_secret("time_secrets", time_secrets_json)\nexcept hopsworks.RestAPIError:\n    print("time_secrets already exists. To update, delete the secret in the UI (https://c.app.hopsworks.ai/account/secrets) and re-run this cell.")\

### Feature group 1: Bike data

In [12]:
bike_fg = fs.get_or_create_feature_group(
    name='bike_data',
    description='How many bikes are available at each station at a given time',
    version=1,
    primary_key=["datetime", "station"],
    event_time="datetime"
)

bike_fg.insert(df_bike)

bike_fg.update_feature_description("num_bikes_available", "Number of bikes available at the station")
bike_fg.update_feature_description("datetime", "Timestamp of the observation")
bike_fg.update_feature_description("station", "Name of the station")

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1207494/fs/1195126/fg/1398091


Uploading Dataframe: 100.00% |██████████| Rows 9279/9279 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: bike_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/bike_data_1_offline_fg_materialization/executions


### Feature group 2: Weather data

In [13]:
weather_fg = fs.get_or_create_feature_group(
    name='weather_data',
    description='Weather data for the city',
    version=1,
    primary_key=["datetime"],
    event_time="datetime"
)

weather_fg.insert(weather_df, write_options={"wait_for_job": True})

# "hourly": ["temperature_2m", "apparent_temperature", "rain", "snowfall", "snow_depth", "wind_speed_10m"],
# "daily": ["sunshine_duration", "daylight_duration", "rain_sum"]

weather_fg.update_feature_description("datetime", "Timestamp of the observation")
weather_fg.update_feature_description("temperature_2m", "Temperature in Celsius")
weather_fg.update_feature_description("apparent_temperature", "Apparent temperature in Celsius")
weather_fg.update_feature_description("rain", "Rain percipitation in mm")
weather_fg.update_feature_description("snowfall", "Snowfall in mm")
weather_fg.update_feature_description("wind_speed_10m", "Wind speed 10 m above ground level")
weather_fg.update_feature_description("daylight_duration", "Daylight duration in seconds")
weather_fg.update_feature_description("rain_sum", "Total rain in mm")

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1207494/fs/1195126/fg/1398092


Uploading Dataframe: 100.00% |██████████| Rows 12456/12456 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: weather_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207494/jobs/named/weather_data_1_offline_fg_materialization/executions
2025-01-21 10:49:13,203 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-01-21 10:53:06,073 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-01-21 10:54:54,659 INFO: Waiting for log aggregation to finish.
2025-01-21 10:58:51,235 INFO: Execution finished successfully.
